In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install torch torchvision
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git
!pip install viam-sdk
# this project mainly use this repo

In [ ]:
from PIL import Image
from lang_sam import LangSAM
model = LangSAM()


final text_encoder_type: bert-base-uncased
Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

def get_mask(model,image,prompt):
    masks, boxes, phrases, logits = model.predict(image, prompt)
    best_mask=masks[:1]
    return best_mask

def plot_image_with_masks(best_masks, image_pil, r, alpha=0.5):

    mask = best_masks.numpy()[0].astype(np.uint8) * 255

    mask_image = Image.fromarray(mask).resize(image_pil.size, Image.NEAREST)
    centroid_x, centroid_y = calculate_centroid(best_masks.numpy()[0])

    draw = ImageDraw.Draw(mask_image)
    image_width = image_pil.size[0]
    central_region_start = int(image_width * ((100 - r) / 2) / 100)
    central_region_end = int(image_width * ((100 + r) / 2) / 100)

    draw.line([(central_region_start, 2), (central_region_start, mask_image.size[1])],fill=120,  width=2)
    draw.line([(central_region_end, 2), (central_region_end, mask_image.size[1])], fill=120 , width=2)
    draw.ellipse([(centroid_x-5, centroid_y-5), (centroid_x+5, centroid_y+5)], fill=1)
    blended = Image.blend(image_pil, mask_image.convert('RGB'), alpha=alpha)

    # plt.clf()
    plt.figure(figsize=(9, 6))
    plt.imshow(blended)
    plt.title('Mask, Decision Boundary, and Centroid')
    plt.axis('off')
    plt.show()

def calculate_centroid(mask):
    y_indices, x_indices = np.where(mask)
    centroid_x = np.mean(x_indices)
    centroid_y = np.mean(y_indices)

    return centroid_x, centroid_y

def plot_and_find_decision(best_masks, image_pil, r, alpha=0.5):
    mask = best_masks.numpy()[0].astype(np.uint8) * 255
    centroid_x, centroid_y = calculate_centroid(mask)
    plot_image_with_masks(best_masks, image_pil, r=r)

    image_width = image_pil.size[0]
    central_region_start = image_width * ((100 - r) / 2) / 100
    central_region_end = image_width * ((100 + r) / 2) / 100

    if centroid_x < central_region_start:
        return 0
    elif centroid_x > central_region_end:
        return 2
    else:
        return 1


# direction = plot_and_find_decision(best_masks, image_pil, r=20)
# plot_image_with_masks(best_masks, image_pil, r=20)
# plot_and_find_decision(best_masks,image_pil,30)

def decide(model,image,prompt):
  mask=get_mask(model,image,prompt)
  return plot_and_find_decision(mask,image,30)


In [ ]:
import asyncio

from viam.robot.client import RobotClient
from viam.rpc.dial import Credentials, DialOptions
from viam.services.vision import VisionClient
from viam.components.camera import Camera
from viam.components.base import Base


async def connect():
    creds = Credentials(
        type="robot-location-secret",
        payload="jeagevngvv75aff0tidbz48t7nm59hl0gl5cut0mm5lslnon")
    opts = RobotClient.Options(
        refresh_interval=0,
        dial_options=DialOptions(credentials=creds)
    )
    return await RobotClient.at_address("stevejin-main.1bftznk8if.viam.cloud", opts)

async def main():
    spinNum = 5         # when turning, spin the motor this much
    straightNum = 300    # when going straight, spin motor this much
    numCycles = 200      # run the loop X times
    vel = 500            # go this fast when moving motor

    # Connect to robot client and set up components
    robot = await connect()
    base = Base.from_robot(robot, "viam_base")
    camera_name = "cam"
    camera = Camera.from_robot(robot, camera_name)
    frame = await camera.get_image(mime_type="image/jpeg")

    # Grab the vision service for the detector
    my_detector = VisionClient.from_robot(robot, "my_color_detector")

    # Main loop. Detect the ball, determine if it's on the left or right, and
    # head that way. Repeat this for numCycles
    for i in range(numCycles):
        # detections = await my_detector.get_detections_from_camera(camera_name)
        frame = await camera.get_image(mime_type="image/jpeg")
        answer = decide(model,frame,PROMPT)
        print(answer)
        if answer == 0:
            print("left")
            await base.spin(spinNum, vel)
        if answer == 1:
            print("center")
            await base.move_straight(straightNum, vel)
        if answer == 2:
            print("right")
            await base.spin(-spinNum, vel)
        # If nothing is detected, nothing moves
        if answer==-1:
            print("searching")
            await base.spin(-spinNum, vel)

    await robot.close()

PROMPT="the monitor"
# PROMPT="the cardboard box"
# PROMPT="the rubbish bin"
await main()
